In [1]:
import io
import os
import tempfile
from pathlib import Path

from rich import print as rprint

from entitysdk.client import Client
from entitysdk.common import ProjectContext
from entitysdk.models import (
    BrainLocation,
    BrainRegion,
    CellMorphology,
    CellMorphologyProtocol,
    Contribution,
    DigitalReconstructionCellMorphologyProtocol,
    MTypeClass,
    MTypeClassification,
    Organization,
    Role,
    Species,
    Strain,
    Subject,
)
from entitysdk.types import (
    CellMorphologyProtocolDesign,
    SlicingDirectionType,
)

### Initialize the entitycore client

In [2]:
entitycore_api_url = "http://127.0.0.1:8000"
project_context = ProjectContext(
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730",
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6",
)
token = os.getenv("ACCESS_TOKEN", "XXX")
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)


# uncomment for staging
# from obi_auth import get_token
# token = get_token(environment="staging")
# Replace this with your vlab project url in staging
# project_context = ProjectContext.from_vlab_url("https://staging.openbraininstitute.org/app/virtual-lab/lab/594fd60d-7a38-436f-939d-500feaa13bba/project/ff89ca07-6613-4922-9ab0-2637221db8b5/home")
# client = Client(environment="staging", project_context=project_context)

## Search for entities

In [3]:
species = client.search_entity(entity_type=Species, query={"name": "Mus musculus"}, limit=10).one()

In [4]:
rprint(species)

Species(
    id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1'),
    update_date=datetime.datetime(2025, 6, 27, 11, 10, 53, 183929, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 6, 27, 11, 10, 53, 183929, tzinfo=TzInfo(UTC)),
    name='Mus musculus',
    taxonomy_id='NCBITaxon:10090'
)

In [5]:
strain = client.search_entity(entity_type=Strain, query={"name": "Cux2-CreERT2"}).one()

In [6]:
rprint(strain)

Strain(
    id=UUID('0e83b9eb-ba0c-4b1d-89aa-3a7f27ed6a31'),
    update_date=datetime.datetime(2025, 6, 27, 11, 10, 56, 853949, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 6, 27, 11, 10, 56, 853949, tzinfo=TzInfo(UTC)),
    name='Cux2-CreERT2',
    taxonomy_id='http://bbp.epfl.ch/neurosciencegraph/ontologies/speciestaxonomy/RBS4I6tyfUBSDt1i0jXLpgN',
    species_id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1')
)

In [7]:
brain_region = client.search_entity(entity_type=BrainRegion, query={"annotation_value": 68}).one()

In [8]:
rprint(brain_region)

BrainRegion(
    id=UUID('72893207-1e8f-48f3-b17b-075b58b9fac5'),
    update_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
    name='Frontal pole, layer 1',
    annotation_value=68,
    acronym='FRP1',
    parent_structure_id=UUID('37fa6e0c-6e6f-4ad6-b4c1-d8e7ffd012b1'),
    hierarchy_id=UUID('e3e70682-c209-4cac-a29f-6fbed82c07cd'),
    color_hex_triplet='268F45'
)

### Create and register a new subject

Subjects should be reused if possible, but they can be created when needed.

In [9]:
subject = Subject(
    name="my-subject-for-morphology",
    description="my-subject-description",
    sex="male",
    species=species,
    strain=strain,
)

subject = client.register_entity(subject)
rprint(subject)

Subject(
    assets=[],
    id=UUID('9fab65ef-89e6-40bc-a675-d21a08fdef4a'),
    update_date=datetime.datetime(2025, 9, 19, 13, 57, 48, 211436, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 9, 19, 13, 57, 48, 211436, tzinfo=TzInfo(UTC)),
    name='my-subject-for-morphology',
    description='my-subject-description',
    type=None,
    created_by=Person(
        id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Admin User',
        given_name=None,
        family_name=None
    ),
    updated_by=Person(
        id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Admin User',
        given_name=None,
        family_name=None
    ),
    authorized_public=False,
    authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
    contributions=None,
    legacy_id=None,
    sex=<Sex.male: 'male'>,
    weight=None,
    age_value=None,
    age_min=None,
    age_max=None,
    age_period=None,
    species=Species(
        id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1'),
        update_date=None,
        creation_date=None,
        name='Mus musculus',
        taxonomy_id='NCBITaxon:10090'
    ),
    strain=Strain(
        id=UUID('0e83b9eb-ba0c-4b1d-89aa-3a7f27ed6a31'),
        update_date=None,
        creation_date=None,
        name='Cux2-CreERT2',
        taxonomy_id='http://bbp.epfl.ch/neurosciencegraph/ontologies/speciestaxonomy/RBS4I6tyfUBSDt1i0jXLpgN',
        species_id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1')
    )
)

### Create and register a new morphology protocol

Morphology protocols should be reused if possible, but they can be created when needed.

In [10]:
morphology_protocol = DigitalReconstructionCellMorphologyProtocol(
  protocol_document="https://example.com/",
  protocol_design=CellMorphologyProtocolDesign.cell_patch,
  slicing_thickness=20.0,
  slicing_direction=SlicingDirectionType.horizontal,
)
morphology_protocol = client.register_entity(morphology_protocol)
rprint(morphology_protocol)

DigitalReconstructionCellMorphologyProtocol(
    assets=[],
    id=UUID('dd180105-ad8f-42bb-a093-17cd6836e9de'),
    update_date=datetime.datetime(2025, 9, 19, 13, 57, 50, 819129, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 9, 19, 13, 57, 50, 819129, tzinfo=TzInfo(UTC)),
    name=None,
    description=None,
    type=<EntityType.cell_morphology_protocol: 'cell_morphology_protocol'>,
    created_by=Person(
        id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Admin User',
        given_name=None,
        family_name=None
    ),
    updated_by=Person(
        id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Admin User',
        given_name=None,
        family_name=None
    ),
    authorized_public=False,
    authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
    contributions=None,
    legacy_id=None,
    protocol_document='https://example.com/',
    protocol_design=<CellMorphologyProtocolDesign.cell_patch: 'cell_patch'>,
    generation_type=<CellMorphologyGenerationType.digital_reconstruction: 'digital_reconstruction'>,
    staining_type=None,
    slicing_thickness=20.0,
    slicing_direction=<SlicingDirectionType.horizontal: 'horizontal'>,
    magnification=None,
    tissue_shrinkage=None,
    corrected_for_shrinkage=None
)

### Create a morphology object

In [15]:
brain_location = BrainLocation(
    x=4101.52490234375,
    y=1173.8499755859375,
    z=4744.60009765625,
)
morphology = CellMorphology(
    cell_morphology_protocol=morphology_protocol,
    name="my-morph",
    description="A morphology",
    subject=subject,
    brain_region=brain_region,
    location=brain_location,
    legacy_id=None,
    authorized_public=False,
)

In [16]:
rprint(morphology)

CellMorphology(
    assets=[],
    id=None,
    update_date=None,
    creation_date=None,
    name='my-morph',
    description='A morphology',
    type=None,
    created_by=None,
    updated_by=None,
    authorized_public=False,
    authorized_project_id=None,
    contributions=None,
    legacy_id=None,
    experiment_date=None,
    contact_email=None,
    subject=Subject(
        assets=[],
        id=UUID('9fab65ef-89e6-40bc-a675-d21a08fdef4a'),
        update_date=datetime.datetime(2025, 9, 19, 13, 57, 48, 211436, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 9, 19, 13, 57, 48, 211436, tzinfo=TzInfo(UTC)),
        name='my-subject-for-morphology',
        description='my-subject-description',
        type=None,
        created_by=Person(
            id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
            update_date=None,
            creation_date=None,
            type='person',
            pref_label='Admin User',
            given_name=None,
            family_name=None
        ),
        updated_by=Person(
            id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
            update_date=None,
            creation_date=None,
            type='person',
            pref_label='Admin User',
            given_name=None,
            family_name=None
        ),
        authorized_public=False,
        authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
        contributions=None,
        legacy_id=None,
        sex=<Sex.male: 'male'>,
        weight=None,
        age_value=None,
        age_min=None,
        age_max=None,
        age_period=None,
        species=Species(
            id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1'),
            update_date=None,
            creation_date=None,
            name='Mus musculus',
            taxonomy_id='NCBITaxon:10090'
        ),
        strain=Strain(
            id=UUID('0e83b9eb-ba0c-4b1d-89aa-3a7f27ed6a31'),
            update_date=None,
            creation_date=None,
            name='Cux2-CreERT2',
            taxonomy_id='http://bbp.epfl.ch/neurosciencegraph/ontologies/speciestaxonomy/RBS4I6tyfUBSDt1i0jXLpgN',
            species_id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1')
        )
    ),
    brain_region=BrainRegion(
        id=UUID('72893207-1e8f-48f3-b17b-075b58b9fac5'),
        update_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
        name='Frontal pole, layer 1',
        annotation_value=68,
        acronym='FRP1',
        parent_structure_id=UUID('37fa6e0c-6e6f-4ad6-b4c1-d8e7ffd012b1'),
        hierarchy_id=UUID('e3e70682-c209-4cac-a29f-6fbed82c07cd'),
        color_hex_triplet='268F45'
    ),
    license=None,
    published_in=None,
    cell_morphology_protocol=DigitalReconstructionCellMorphologyProtocol(
        assets=[],
        id=UUID('dd180105-ad8f-42bb-a093-17cd6836e9de'),
        update_date=datetime.datetime(2025, 9, 19, 13, 57, 50, 819129, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 9, 19, 13, 57, 50, 819129, tzinfo=TzInfo(UTC)),
        name=None,
        description=None,
        type=<EntityType.cell_morphology_protocol: 'cell_morphology_protocol'>,
        created_by=Person(
            id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
            update_date=None,
            creation_date=None,
            type='person',
            pref_label='Admin User',
            given_name=None,
            family_name=None
        ),
        updated_by=Person(
            id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
            update_date=None,
            creation_date=None,
            type='person',
            pref_label='Admin User',
            given_name=None,
            family_name=None
        ),
        authorized_public=False,
        authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
        contributions=None,
        legacy_id

## Register morphology

In [17]:
registered = client.register_entity(entity=morphology)

In [18]:
rprint(registered)

CellMorphology(
    assets=[],
    id=UUID('b8c2cddd-2e55-4f15-bf89-75e128a01eb4'),
    update_date=datetime.datetime(2025, 9, 19, 14, 2, 10, 106808, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 9, 19, 14, 2, 10, 106808, tzinfo=TzInfo(UTC)),
    name='my-morph',
    description='A morphology',
    type=<EntityType.cell_morphology: 'cell_morphology'>,
    created_by=Person(
        id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Admin User',
        given_name=None,
        family_name=None
    ),
    updated_by=Person(
        id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Admin User',
        given_name=None,
        family_name=None
    ),
    authorized_public=False,
    authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
    contributions=[],
    legacy_id=None,
    experiment_date=None,
    contact_email=None,
    subject=Subject(
        assets=[],
        id=UUID('9fab65ef-89e6-40bc-a675-d21a08fdef4a'),
        update_date=None,
        creation_date=None,
        name='my-subject-for-morphology',
        description='my-subject-description',
        type=None,
        created_by=None,
        updated_by=None,
        authorized_public=False,
        authorized_project_id=None,
        contributions=None,
        legacy_id=None,
        sex=<Sex.male: 'male'>,
        weight=None,
        age_value=None,
        age_min=None,
        age_max=None,
        age_period=None,
        species=Species(
            id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1'),
            update_date=None,
            creation_date=None,
            name='Mus musculus',
            taxonomy_id='NCBITaxon:10090'
        ),
        strain=Strain(
            id=UUID('0e83b9eb-ba0c-4b1d-89aa-3a7f27ed6a31'),
            update_date=None,
            creation_date=None,
            name='Cux2-CreERT2',
            taxonomy_id='http://bbp.epfl.ch/neurosciencegraph/ontologies/speciestaxonomy/RBS4I6tyfUBSDt1i0jXLpgN',
            species_id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1')
        )
    ),
    brain_region=BrainRegion(
        id=UUID('72893207-1e8f-48f3-b17b-075b58b9fac5'),
        update_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 6, 27, 11, 10, 27, 186088, tzinfo=TzInfo(UTC)),
        name='Frontal pole, layer 1',
        annotation_value=68,
        acronym='FRP1',
        parent_structure_id=UUID('37fa6e0c-6e6f-4ad6-b4c1-d8e7ffd012b1'),
        hierarchy_id=UUID('e3e70682-c209-4cac-a29f-6fbed82c07cd'),
        color_hex_triplet='268F45'
    ),
    license=None,
    published_in=None,
    cell_morphology_protocol=DigitalReconstructionCellMorphologyProtocol(
        assets=[],
        id=UUID('dd180105-ad8f-42bb-a093-17cd6836e9de'),
        update_date=None,
        creation_date=None,
        name=None,
        description=None,
        type=<EntityType.cell_morphology_protocol: 'cell_morphology_protocol'>,
        created_by=None,
        updated_by=None,
        authorized_public=False,
        authorized_project_id=None,
        contributions=None,
        legacy_id=None,
        protocol_document='https://example.com/',
        protocol_design=<CellMorphologyProtocolDesign.cell_patch: 'cell_patch'>,
        generation_type=<CellMorphologyGenerationType.digital_reconstruction: 'digital_reconstruction'>,
        staining_type=None,
        slicing_thickness=20.0,
        slicing_direction=<SlicingDirectionType.horizontal: 'horizontal'>,
        magnification=None,
        tissue_shrinkage=None,
        corrected_for_shrinkage=None
    ),
    location=BrainLocation(x=4101.52490234375, y=1173.8499755859375, z=4744.60009765625),
    mtypes=[]
)

## Add mtype to morphology

In [19]:
mtype = client.search_entity(entity_type=MTypeClass, query={"pref_label": "L5_PC"}).one()

mtype_classification = MTypeClassification(
    mtype_class_id=mtype.id, entity_id=registered.id, authorized_public=True
)

mtype_classification = client.register_entity(mtype_classification)

## Add contribution to morphology

In [20]:
agent = client.search_entity(entity_type=Organization, limit=1).one()
role = client.search_entity(entity_type=Role, limit=1).one()
contribution = Contribution(
    agent=agent,
    role=role,
    entity=registered,
)
contribution = client.register_entity(contribution)

In [21]:
rprint(contribution)

Contribution(
    id=UUID('5424af0d-9e95-404f-a2ba-6310e4de5320'),
    update_date=datetime.datetime(2025, 9, 19, 14, 2, 43, 341947, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 9, 19, 14, 2, 43, 341947, tzinfo=TzInfo(UTC)),
    agent=Organization(
        id=UUID('c8c69069-134b-4cd3-a1cf-4f589f8e4ce0'),
        update_date=None,
        creation_date=None,
        type='organization',
        pref_label='Yale School of Medicine',
        alternative_name=''
    ),
    role=Role(
        id=UUID('5db149c5-6bcf-42df-aea5-f075cac49ff2'),
        update_date=datetime.datetime(2025, 6, 27, 11, 12, 40, 177373, tzinfo=TzInfo(UTC)),
        creation_date=datetime.datetime(2025, 6, 27, 11, 12, 40, 177373, tzinfo=TzInfo(UTC)),
        name='creator role',
        role_id='http://purl.obolibrary.org/obo/CRO_0000107'
    ),
    entity=Entity(
        assets=[],
        id=UUID('b8c2cddd-2e55-4f15-bf89-75e128a01eb4'),
        update_date=None,
        creation_date=None,
        name=None,
        description=None,
        type=<EntityType.cell_morphology: 'cell_morphology'>,
        created_by=None,
        updated_by=None,
        authorized_public=False,
        authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
        contributions=None,
        legacy_id=None
    )
)

## Upload assets

In [22]:
with tempfile.TemporaryDirectory() as tdir:
    file1 = Path(tdir, "morph.h5")
    file1.write_text("h5")

    file2 = Path(tdir, "morph.swc")
    file2.write_text("swc")

    # use a filepath to register first asset
    asset1 = client.upload_file(
        entity_id=registered.id,
        entity_type=CellMorphology,
        file_path=file1,
        file_content_type="application/x-hdf5",
        asset_label="morphology",
    )
    rprint(asset1)

    # use an in-memory buffer to upload second asset
    buffer = io.BytesIO(b"morph bytes buffer")

    asset2 = client.upload_content(
        entity_id=registered.id,
        entity_type=CellMorphology,
        file_content=buffer,
        file_name="buffer.swc",
        file_content_type="application/swc",
        asset_label="morphology",
    )
    rprint(asset2)

Asset(
    id=UUID('a17b92c1-30ae-4c94-b2f6-f7ad65215e90'),
    update_date=None,
    creation_date=None,
    path='morph.h5',
    full_path='private/a98b7abc-fc46-4700-9e3d-37137812c730/0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6/assets/cell_morpho
logy/b8c2cddd-2e55-4f15-bf89-75e128a01eb4/morph.h5',
    is_directory=False,
    content_type=<ContentType.application_x_hdf5: 'application/x-hdf5'>,
    size=2,
    sha256_digest='acea77fd89c7fb586c8c7c612b53634c4571a40cf8b1bcecb002311210423f57',
    status=<AssetStatus.created: 'created'>,
    meta={},
    label=<AssetLabel.morphology: 'morphology'>
)

Asset(
    id=UUID('ddd90e1f-5120-45bf-805c-2e5a8ce46ffd'),
    update_date=None,
    creation_date=None,
    path='buffer.swc',
    full_path='private/a98b7abc-fc46-4700-9e3d-37137812c730/0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6/assets/cell_morpho
logy/b8c2cddd-2e55-4f15-bf89-75e128a01eb4/buffer.swc',
    is_directory=False,
    content_type=<ContentType.application_swc: 'application/swc'>,
    size=18,
    sha256_digest='47ddc1b6e05dcbfbd2db9dcec4a49d83c6f9f10ad595649bacdcb629671fd954',
    status=<AssetStatus.created: 'created'>,
    meta={},
    label=<AssetLabel.morphology: 'morphology'>
)

## Retrieve it

In [23]:
# with assets and mtypes
fetched = client.get_entity(entity_id=registered.id, entity_type=CellMorphology)

## Download asset

In [24]:
downloaded = client.download_assets(
    fetched,
    selection={"content_type": "application/swc"},
    output_path="./my-file.swc",
).one()

content = client.download_content(
    entity_id=fetched.id, entity_type=type(fetched), asset_id=downloaded.asset.id
)

print(content)
print(Path("my-file.h5").read_text())

b'morph bytes buffer'
morph bytes buffer


## Delete asset

In [25]:
for asset in fetched.assets:
    if asset.content_type == "application/swc":
        print("Deleting asset ", asset.id)
        deleted_asset = client.delete_asset(
            entity_id=fetched.id,
            entity_type=type(registered),
            asset_id=asset.id,
        )
        break

rprint(deleted_asset)

fetched = client.get_entity(entity_id=registered.id, entity_type=CellMorphology)
rprint(fetched.assets)

Deleting asset  ddd90e1f-5120-45bf-805c-2e5a8ce46ffd


Asset(
    id=UUID('ddd90e1f-5120-45bf-805c-2e5a8ce46ffd'),
    update_date=None,
    creation_date=None,
    path='buffer.swc',
    full_path='private/a98b7abc-fc46-4700-9e3d-37137812c730/0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6/assets/cell_morpho
logy/b8c2cddd-2e55-4f15-bf89-75e128a01eb4/buffer.swc',
    is_directory=False,
    content_type=<ContentType.application_swc: 'application/swc'>,
    size=18,
    sha256_digest='47ddc1b6e05dcbfbd2db9dcec4a49d83c6f9f10ad595649bacdcb629671fd954',
    status=<AssetStatus.deleted: 'deleted'>,
    meta={},
    label=<AssetLabel.morphology: 'morphology'>
)

[
    Asset(
        id=UUID('a17b92c1-30ae-4c94-b2f6-f7ad65215e90'),
        update_date=None,
        creation_date=None,
        path='morph.h5',
        full_path='private/a98b7abc-fc46-4700-9e3d-37137812c730/0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6/assets/cell_mo
rphology/b8c2cddd-2e55-4f15-bf89-75e128a01eb4/morph.h5',
        is_directory=False,
        content_type=<ContentType.application_x_hdf5: 'application/x-hdf5'>,
        size=2,
        sha256_digest='acea77fd89c7fb586c8c7c612b53634c4571a40cf8b1bcecb002311210423f57',
        status=<AssetStatus.created: 'created'>,
        meta={},
        label=<AssetLabel.morphology: 'morphology'>
    )
]

## Search it

In [26]:
hits = client.search_entity(
    entity_type=CellMorphology,
    query={"name__ilike": "my-morph", "page": 1, "page_size": 2},
    limit=None,
).all()

print("Number of results: ", len(hits))

Number of results:  2


In [27]:
rprint(hits[0])

CellMorphology(
    assets=[
        Asset(
            id=UUID('a17b92c1-30ae-4c94-b2f6-f7ad65215e90'),
            update_date=None,
            creation_date=None,
            path='morph.h5',
            full_path='private/a98b7abc-fc46-4700-9e3d-37137812c730/0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6/assets/cel
l_morphology/b8c2cddd-2e55-4f15-bf89-75e128a01eb4/morph.h5',
            is_directory=False,
            content_type=<ContentType.application_x_hdf5: 'application/x-hdf5'>,
            size=2,
            sha256_digest='acea77fd89c7fb586c8c7c612b53634c4571a40cf8b1bcecb002311210423f57',
            status=<AssetStatus.created: 'created'>,
            meta={},
            label=<AssetLabel.morphology: 'morphology'>
        )
    ],
    id=UUID('b8c2cddd-2e55-4f15-bf89-75e128a01eb4'),
    update_date=datetime.datetime(2025, 9, 19, 14, 2, 10, 106808, tzinfo=TzInfo(UTC)),
    creation_date=datetime.datetime(2025, 9, 19, 14, 2, 10, 106808, tzinfo=TzInfo(UTC)),
    name='my-morph',
    description='A morphology',
    type=<EntityType.cell_morphology: 'cell_morphology'>,
    created_by=Person(
        id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Admin User',
        given_name=None,
        family_name=None
    ),
    updated_by=Person(
        id=UUID('ab00247a-775d-4367-b122-693edc2aecbe'),
        update_date=None,
        creation_date=None,
        type='person',
        pref_label='Admin User',
        given_name=None,
        family_name=None
    ),
    authorized_public=False,
    authorized_project_id=UUID('0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6'),
    contributions=[
        Contribution(
            id=UUID('5424af0d-9e95-404f-a2ba-6310e4de5320'),
            update_date=None,
            creation_date=None,
            agent=Organization(
                id=UUID('c8c69069-134b-4cd3-a1cf-4f589f8e4ce0'),
                update_date=None,
                creation_date=None,
                type='organization',
                pref_label='Yale School of Medicine',
                alternative_name=''
            ),
            role=Role(
                id=UUID('5db149c5-6bcf-42df-aea5-f075cac49ff2'),
                update_date=datetime.datetime(2025, 6, 27, 11, 12, 40, 177373, tzinfo=TzInfo(UTC)),
                creation_date=datetime.datetime(2025, 6, 27, 11, 12, 40, 177373, tzinfo=TzInfo(UTC)),
                name='creator role',
                role_id='http://purl.obolibrary.org/obo/CRO_0000107'
            ),
            entity=None
        )
    ],
    legacy_id=None,
    experiment_date=None,
    contact_email=None,
    subject=Subject(
        assets=[],
        id=UUID('9fab65ef-89e6-40bc-a675-d21a08fdef4a'),
        update_date=None,
        creation_date=None,
        name='my-subject-for-morphology',
        description='my-subject-description',
        type=None,
        created_by=None,
        updated_by=None,
        authorized_public=False,
        authorized_project_id=None,
        contributions=None,
        legacy_id=None,
        sex=<Sex.male: 'male'>,
        weight=None,
        age_value=None,
        age_min=None,
        age_max=None,
        age_period=None,
        species=Species(
            id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1'),
            update_date=None,
            creation_date=None,
            name='Mus musculus',
            taxonomy_id='NCBITaxon:10090'
        ),
        strain=Strain(
            id=UUID('0e83b9eb-ba0c-4b1d-89aa-3a7f27ed6a31'),
            update_date=None,
            creation_date=None,
            name='Cux2-CreERT2',
            taxonomy_id='http://bbp.epfl.ch/neurosciencegraph/ontologies/speciestaxonomy/RBS4I6tyfUBSDt1i0jXLpgN',
            species_id=UUID('b7ad4cca-4ac2-4095-9781-37fb68fe9ca1')
        )
    ),
    brain_region=BrainRegion(
        id=UUID('72893207-1e8f-48f3-b17b-075b58b9fac5'),
        update_date=datetime.datetime(2025, 6, 

## Register existing data from an external cloud storage

In [ ]:
## register a file
# asset = client.register_asset(
#     entity_id=registered.id,
#     entity_type=CellMorphology,
#     name="my-morphology.h5",
#     storage_path="path/to/morph.h5",
#     storage_type="aws_s3_open",
#     is_directory=False,
#     content_type="application/x-hdf5",
#     asset_label="morphology",
# )

In [ ]:
## register a directory
# asset = client.register_asset(
#     entity_id=registered.id,
#     entity_type=Circuit,
#     name="sonata_circuit",
#     storage_path="path/to/circuit/directory",
#     storage_type="aws_s3_open",
#     is_directory=True,
#     content_type="application/vnd.directory",
#     asset_label="sonata_circuit",
# )